<a href="https://colab.research.google.com/github/ravi-prakash1907/Machine-Learning-for-Cyber-Security/blob/main/Labs/deepNN_lab5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment-2 : Tomato Leaf Disease Prediction  

_(by Transfer Learning using VGG-16 Network)_

## Initial Set-Up

In [ ]:
# information about the 
!nvidia-smi

Wed Jun 16 10:11:39 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install tensorflow-gpu

     |████████████████████████████████| 454.3MB 41kB/s 


## Data

instead of just inception-v3, implement with cgg-16 as well

In [24]:
## required libraries
import tensorflow as tf
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16 ## check it's arch, a 42 layer layer net
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from glob import glob
import matplotlib.pyplot as plt

import numpy as np
import sys

import pandas as pd
print(tf.__version__)

from google.colab import drive

2.5.0


In [ ]:
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
dataDir = 'drive/MyDrive/git/Datasets/tomato/'

In [ ]:
## see if we are at right loc
!ls drive/MyDrive/git/Datasets/tomato/

train  valid


## work

In [ ]:
## will need to resize all images (if ot so) 
IMAGE_SIZE = [224,224]

In [ ]:
train_path = dataDir+'train'
valid_path = dataDir+'valid'

_importing the inception-v3 lib. & adding preprocessing layer to the front of inception v3_  

_here it's done using imagenet weights_

In [ ]:
inception = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

58892288/58889256 [==============================] - 1s 0us/step


In [ ]:
# don't traing existing weights
for layer in inception.layers:
  layer.trainable = False

In [ ]:

## for getting number of unique output class
folders = glob(train_path + '/*')

In [ ]:
len(folders), folders

(10,
 ['drive/MyDrive/git/Datasets/tomato/train/Tomato___Leaf_Mold',
  'drive/MyDrive/git/Datasets/tomato/train/Tomato___Tomato_mosaic_virus',
  'drive/MyDrive/git/Datasets/tomato/train/Tomato___Early_blight',
  'drive/MyDrive/git/Datasets/tomato/train/Tomato___Target_Spot',
  'drive/MyDrive/git/Datasets/tomato/train/Tomato___Late_blight',
  'drive/MyDrive/git/Datasets/tomato/train/Tomato___healthy',
  'drive/MyDrive/git/Datasets/tomato/train/Tomato___Bacterial_spot',
  'drive/MyDrive/git/Datasets/tomato/train/Tomato___Tomato_Yellow_Leaf_Curl_Virus',
  'drive/MyDrive/git/Datasets/tomato/train/Tomato___Septoria_leaf_spot',
  'drive/MyDrive/git/Datasets/tomato/train/Tomato___Spider_mites Two-spotted_spider_mite'])

_so, we have 10 classes_

In [ ]:
# adding layers
x = Flatten()(inception.output)

In [ ]:
prediction = Dense(len(folders), activation='softmax')(x)

## creating a model object
model = Model(inputs=inception.input, outputs=prediction)

In [ ]:
# view the structure of the model
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
## using the imgages data generator 
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range=0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
## training  ---  providing same target size as that for image size
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (224,224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 18355 images belonging to 10 classes.


In [ ]:
test_set = test_datagen.flow_from_directory(valid_path,
                                                 target_size = (224,224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 4585 images belonging to 10 classes.


In [20]:
## fit the model - a time consuming process
model = tf.keras.models.load_model('/content/drive/MyDrive/my_model.h5')#0 done
for i in range(10):
  r = model.fit(
      training_set,
      validation_data=test_set,
      steps_per_epoch=100, #0,
      epochs=1, #10,
      validation_steps=100
      #steps_per_epoch=len(training_set),
      #validation_steps=len(test_set)
      )
  model.save('/content/drive/MyDrive/my_model.h5')
  model = tf.keras.models.load_model('/content/drive/MyDrive/my_model.h5')


100/100 [==============================] - 3204s 32s/step - loss: 0.3567 - accuracy: 0.8781 - val_loss: 0.4892 - val_accuracy: 0.8438


**Here model can be seen running only 3 times because, it was ran 7 more times prior to this final execution.**

#### Performance

In [21]:
## model accuracy
print("Accuracy: ",r.history['accuracy'])

Accuracy:  [0.878125011920929]
